### 3.4) Multiplicación de matrices en GPU con Numpy

In [ ]:
import sys
import numpy as np

# Tamaño de la matriz:
# Si el usuario pasa un argumento por línea de comandos, lo usamos como n.
# Si no, usamos un valor por defecto para ejecutar el notebook.
if len(sys.argv) > 1:
    n = int(sys.argv[1])
    print("Tamaño de las matrices (n) leído por línea de comandos:", n)
else:
    n = 7000
    print("Tamaño de las matrices (n) por defecto:", n)
    
A = np.random.rand(n, n).astype(np.float32)
B = np.random.rand(n, n).astype(np.float32)

C = np.dot(A, B)  # warm-up and Matrix multiplication

%timeit -r 2 -o np.dot(A, B)

print(f"Result shape (Numpy): {C.shape}")
print(f"Result type (Numpy): {C.dtype}")

### Multiplicación de matrices en GPU con PyTorch

In [ ]:
import torch

# Seleccionamos el dispositivo: GPU si está disponible (bohr-gpu), si no CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando dispositivo:", device)

# Usamos el mismo n que se ha leído con sys.argv (o 7000 por defecto)
print("Tamaño de las matrices (n) en PyTorch:", n)

# Creamos las matrices directamente en el dispositivo (idealmente en la GPU)
A_t = torch.rand((n, n), dtype=torch.float32, device=device)
B_t = torch.rand((n, n), dtype=torch.float32, device=device)

# Multiplicación de matrices con PyTorch (warm-up)
C_t = torch.matmul(A_t, B_t)

def matmul_torch():
    C = torch.matmul(A_t, B_t)
    if device.type == "cuda":
        torch.cuda.synchronize()
    return C

print("Midiendo tiempo de torch.matmul(A_t, B_t)...")
%timeit -r 2 -o matmul_torch()

print(f"Result shape (PyTorch): {C_t.shape}")
print(f"Result type (PyTorch): {C_t.dtype}")
print(f"Stored on device: {C_t.device}")

### 3.4) Multiplicación de matrices con Numpy y PyTorch en GPU (resultados - extra)

Para esta actividad extra se ha modificado el notebook original de multiplicación de matrices para permitir que el tamaño `n` de las matrices cuadradas pueda ser proporcionado por el usuario desde la línea de comandos. De este modo, al lanzar el notebook mediante `sbatch` e `ipython`, es posible seleccionar dinámicamente el tamaño de la operación, lo que facilita experimentar con distintos valores de `n` sin editar el código.

Una vez incorporada esta mejora, se ha comparado la multiplicación de matrices en NumPy (CPU) con su equivalente en PyTorch utilizando la GPU del nodo `bohr-gpu`. A continuación se resumen los resultados obtenidos para dos tamaños de matriz.

---

#### Caso 1: n = 4000

- **NumPy (CPU)**  
  Tiempo medio: **151 ms**

- **PyTorch (GPU)**  
  Tiempo medio: **9.89 ms**

- **Aceleración aproximada:** ~15×  
  PyTorch realiza toda la operación en la GPU (`cuda:0`), manteniendo el resultado en `float32` y acelerando notablemente el cálculo frente a NumPy.

---

#### Caso 2: n = 7000

- **NumPy (CPU)**  
  Tiempo medio: **569 ms**

- **PyTorch (GPU)**  
  Tiempo medio: **50.4 ms**

- **Aceleración aproximada:** ~11×  
  Para tamaños grandes, la GPU muestra un rendimiento muy superior debido a su capacidad de procesamiento masivamente paralelo.

---

### Conclusión

La incorporación del parámetro `n` a través de la línea de comandos permite ajustar fácilmente el tamaño de las matrices durante la ejecución en `bohr-gpu`, sin necesidad de modificar el código del notebook. Esto facilita la evaluación comparativa entre CPU y GPU para distintos volúmenes de datos.

Los resultados muestran que, para operaciones de multiplicación de matrices de gran tamaño, la ejecución en GPU mediante PyTorch presenta tiempos de cálculo sustancialmente inferiores a los obtenidos con NumPy en CPU. En los casos probados (n = 4000 y n = 7000), PyTorch reduce el tiempo total de la operación en aproximadamente un orden de magnitud, manteniendo la precisión numérica y el formato de datos (`float32`). Esta diferencia se debe al aprovechamiento del paralelismo masivo de la GPU en operaciones intensivas de álgebra lineal.

En conjunto, los experimentos ilustran cómo PyTorch puede emplearse como una alternativa eficiente para realizar cálculos matriciales a gran escala cuando se dispone de hardware acelerador.
